# BabyDragon Threads
The `BaseThread` class is a utility class for managing the memory thread of a conversation and keeping track of the total number of tokens. You can use this class to store, find, and manage conversation messages. The memory thread can have a maximum token limit, or it can be unlimited. After the limit is reached no more messages can be added.

## Initialization
The `__init__()` method initializes the `BaseThread` instance with a name, maximum token limit, and a tokenizer. By default, the name is set to "memory", the maximum token limit is set to None (unlimited), and the tokenizer is set to the tiktoken encoding for the 'gpt-3.5-turbo' model.

In [1]:
from babydragon.memory.threads.base_thread import BaseThread
thread = BaseThread(name='memory_thread', max_memory= None)

## Add and Remove Messages
Use the `add_message()` method to add a message to the memory thread. The message should be a dictionary containing the role and content of the message. If the total tokens in the memory thread would exceed the maximum token limit after adding the message, the message will not be added.

In [2]:
message = {"role": "user", "content": "Hello, how are you?"}
thread.add_message(message)
print(thread.memory_thread)

[{'role': 'user', 'content': 'Hello, how are you?'}]


To remove a message from the memory thread, use the `remove_message()` method. You can either provide the message dictionary:


In [3]:
thread.remove_message(message_dict=message)
print(thread.memory_thread)

[]


or the `idx` of the message in the memory thread:

In [4]:
message = {"role": "user", "content": "Hello, how are you?"}
thread.add_message(message)
thread.remove_message(idx=0)
print(thread.memory_thread)

[]


Let's add a few more messages to use in the next examples:

In [5]:
thread = BaseThread(name='memory_thread', max_memory= None)
thread.add_message({"role": "user", "content": "Hello, how are you?"})
thread.add_message({"role": "assistant", "content": "I'm fine, thanks."})
thread.add_message({"role": "user", "content": "What's your name?"})
thread.add_message({"role": "assistant", "content": "My name is BabyDragon."})
thread.add_message({"role": "user", "content": "Nice to meet you."})
thread.add_message({"role": "assistant", "content": "Nice to meet you too."})
thread.add_message({"role": "user", "content": "Hello, how are you?"})
thread.add_message({"role": "assistant", "content": "Hello, how are you?"})

for memory in thread.memory_thread:
    print(memory)


{'role': 'user', 'content': 'Hello, how are you?'}
{'role': 'assistant', 'content': "I'm fine, thanks."}
{'role': 'user', 'content': "What's your name?"}
{'role': 'assistant', 'content': 'My name is BabyDragon.'}
{'role': 'user', 'content': 'Nice to meet you.'}
{'role': 'assistant', 'content': 'Nice to meet you too.'}
{'role': 'user', 'content': 'Hello, how are you?'}
{'role': 'assistant', 'content': 'Hello, how are you?'}


## Find Messages
The class provides several methods for finding messages in the memory thread, such as:

`find_message("string" or Dict, role)`: Finds a message based on exaxt match of the content or the message dictionary.


In [6]:
search_results = thread.find_message("Hello, how are you?", role = None)
for result in search_results:
    print(result)


{'idx': 0, 'message_dict': {'role': 'user', 'content': 'Hello, how are you?'}}
{'idx': 6, 'message_dict': {'role': 'user', 'content': 'Hello, how are you?'}}
{'idx': 7, 'message_dict': {'role': 'assistant', 'content': 'Hello, how are you?'}}


You can further filter the search by the `role`:

In [7]:
search_results = thread.find_message("Hello, how are you?", role = "assistant")
for result in search_results:
    print(result)

{'idx': 7, 'message_dict': {'role': 'assistant', 'content': 'Hello, how are you?'}}


Or you can directly pass a dictionary defining both the `content` and the `role`:

In [8]:
message = {"role": "user", "content": "Hello, how are you?"}
search_results = thread.find_message(message)
for result in search_results:
    print(result)

{'idx': 0, 'message_dict': {'role': 'user', 'content': 'Hello, how are you?'}}
{'idx': 6, 'message_dict': {'role': 'user', 'content': 'Hello, how are you?'}}


`find_role(role:str)`: Finds all messages with a specific role in the memory thread.

In [9]:
user_messages = thread.find_role("user")
for message in user_messages:
    print(message)


{'idx': 0, 'message_dict': {'role': 'user', 'content': 'Hello, how are you?'}}
{'idx': 2, 'message_dict': {'role': 'user', 'content': "What's your name?"}}
{'idx': 4, 'message_dict': {'role': 'user', 'content': 'Nice to meet you.'}}
{'idx': 6, 'message_dict': {'role': 'user', 'content': 'Hello, how are you?'}}


`last_message()`: Gets the last message as a dictionary in the memory thread with a specific role.

In [10]:
# Get the last message with a specific role (e.g., "user")
last_user_message = thread.last_message(role=None)
print(last_user_message)

{'role': 'assistant', 'content': 'Hello, how are you?'}


`first_message()`: Gets the first message in the memory thread with a specific role.

In [11]:
first_user_message = thread.first_message(role="user")
print(first_user_message)

{'role': 'user', 'content': 'Hello, how are you?'}


`messages_before()`: Gets all messages before a specific message in the memory thread with a specific role.

In [12]:
message = {'role': 'assistant', 'content': 'My name is BabyDragon.'}

messages_before = thread.messages_before(message, role='user')
for message in messages_before:
    print(message)

{'role': 'user', 'content': 'Hello, how are you?'}
{'role': 'user', 'content': "What's your name?"}


`messages_after()`: Gets all messages after a specific message in the memory thread with a specific role.

In [13]:
messages_after = thread.messages_after(message, role=None)
for message in messages_after:
    print(message)

{'role': 'user', 'content': 'Hello, how are you?'}
{'role': 'assistant', 'content': "I'm fine, thanks."}


`messages_between()`: Gets all messages between two specific messages in the memory thread with a specific role.
Here are examples of how to use these methods:

In [14]:
messages_between = thread.messages_between(first_user_message, last_user_message, role=None)
for message in messages_between:
    print(message)

{'role': 'assistant', 'content': "I'm fine, thanks."}
{'role': 'user', 'content': "What's your name?"}
{'role': 'assistant', 'content': 'My name is BabyDragon.'}
{'role': 'user', 'content': 'Nice to meet you.'}
{'role': 'assistant', 'content': 'Nice to meet you too.'}


## Token Bound History
The following method is used to get the most recent messages from the memory thread within a specified token limit.

`token_bound_history(max_tokens: int, max_history=None, role: Union[str, None] = None)`
This method returns a tuple of messages and their indices that fit within the max_tokens limit, from the most recent messages in the memory thread with a specific role. If the role parameter is not provided, it will return messages with any role. The max_history parameter, if provided, limits the search to the most recent max_history messages.

In [15]:
for message in thread.memory_thread:
    tokens = len(thread.tokenizer.encode(message['content']))
    tokens = tokens+ 6 # 6 tokens for the special tokens
    print(f"Message: {message['content']}, Tokens: {tokens}")

Message: Hello, how are you?, Tokens: 12
Message: I'm fine, thanks., Tokens: 12
Message: What's your name?, Tokens: 11
Message: My name is BabyDragon., Tokens: 12
Message: Nice to meet you., Tokens: 11
Message: Nice to meet you too., Tokens: 12
Message: Hello, how are you?, Tokens: 12
Message: Hello, how are you?, Tokens: 12


In [17]:
messages, indices = thread.token_bound_history(36, max_history=10, role=None)
for message in messages:
    print(message)

{'role': 'assistant', 'content': 'Hello, how are you?'}
{'role': 'user', 'content': 'Hello, how are you?'}
{'role': 'assistant', 'content': 'Nice to meet you too.'}
